In [1]:
from tensorflow.keras.applications import ConvNeXtSmall
from tl_tools import *
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import pandas as pd

In [2]:
setup_mixed_precision()
train_dir = '/Users/pimpijnenburg/Desktop/Thesis/USTC_SmokeRS_dataset/data/USTC_SmokeRS/processed/train'

train_datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=True ,vertical_flip=True)
train =train_datagen.flow_from_directory(train_dir, color_mode= 'rgb', batch_size = 16, shuffle= True, seed = 1, target_size=(256, 256))


X_train, y_train, X_val, y_val = train_val_split(train, val_split= 0.3)

Mixed precision policy set to: mixed_float16

Found 4980 images belonging to 6 classes.
Number of batches in the training data: 312
Batch size of a single batch 16
Number of samples in the training dataset 4980

Number of training data batches with val split of 0.3: 219
Number of validation data batches: 93

Shape of image training set: (3504, 256, 256, 3)
Shape of image validation set: (1476, 256, 256, 3)

Shape of label training set: (3504, 6)
Shape of label validation set: (1476, 6)


In [3]:
test_dir = '/Users/pimpijnenburg/Desktop/Thesis/USTC_SmokeRS_dataset/data/USTC_SmokeRS/processed/test'
test_datagen = ImageDataGenerator(rescale=1./255)
test = test_datagen.flow_from_directory(test_dir, color_mode= 'rgb', batch_size = 16, shuffle= True, seed = 1, target_size=(256, 256))
X_test, y_test = test_splits(test)

Found 1245 images belonging to 6 classes.
Number of batches in the test data: 78
Batch size of a single batch 16
Number of samples in the test dataset 1245

Shape of image test set: (1245, 256, 256, 3)

Shape of label test set: (1245, 6)


In [4]:
pooling = 'avg'
convnext = ConvNeXtSmall(weights= 'imagenet',include_top= False, input_shape= (256, 256, 3), pooling = pooling)
convnext.trainable = False

model = fc_layers(convnext, name = 'ConvNextSmall')

In [ ]:
model.compile(optimizer=Adam(learning_rate = 0.001, clipnorm= 1.0),
                     loss='categorical_crossentropy', 
                     metrics=['accuracy', 'F1Score'])

early_stopping = EarlyStopping(monitor='val_loss', patience=15,restore_best_weights=True, start_from_epoch=20)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=1e-6)

history =  model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=50, verbose=1, 
                            callbacks= [early_stopping, reduce_lr])

Epoch 1/50


I0000 00:00:1730123808.288523  247287 service.cc:145] XLA service 0x3a6af9fc0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1730123808.288705  247287 service.cc:153]   StreamExecutor device (0): Host, Default Version
I0000 00:00:1730123808.388621  247287 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


  3/110 ━━━━━━━━━━━━━━━━━━━━ 22:23 13s/step - F1Score: 0.2173 - accuracy: 0.1979 - loss: 2.2206

In [4]:
pred = tf.argmax(convnext.predict(X_test), axis = 1).numpy()
y_true = tf.argmax(y_test, axis=1).numpy()

NameError: name 'convnext' is not defined

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

class_labels = list(test.class_indices.keys())
confusion_matrix = confusion_matrix(pred, y_true)


plt.figure(figsize=(10, 8))
sns.heatmap(confusion_matrix, annot=True, fmt='d', cmap='Blues', 
            xticklabels=class_labels, yticklabels=class_labels)
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

In [ ]:
n_classes = len(class_labels)
FP = confusion_matrix.sum(axis=0) - np.diag(confusion_matrix)
FN = confusion_matrix.sum(axis=1) - np.diag(confusion_matrix)
TP = np.diag(confusion_matrix)
TN = confusion_matrix.sum() - (FP + FN + TP)

metrics = ['Precision', 'Recall', 'F1-score', 'False Negative Rate']

df_metrics = pd.DataFrame(index=metrics, columns=class_labels)

for i in range(n_classes):
    df_metrics.iloc[0, i] = TP[i] / (TP[i] + FP[i])
    df_metrics.iloc[1, i] = TP[i] / (TP[i] + FN[i])
    df_metrics.iloc[2, i] = 2 * TP[i] / (2 * TP[i] + FP[i] + FN[i])
    df_metrics.iloc[3, i] = FN[i] / (FN[i] + TP[i])  # False Negative Rate

def round_fn(input): 
    return round(input, 4)

df_metrics = df_metrics.map(round_fn)
df_metrics

In [ ]:
from sklearn.metrics import accuracy_score

print(f'Accuracy Score: {accuracy_score(pred, y_true):.4f}')
for index, row in df_metrics.iterrows(): 
    row_mean = row.mean()
    print(f'Average {index} Score: {row_mean:.4f}')

In [ ]:
feature_extractor = Model(inputs=convnext.input, outputs=convnext.get_layer('global_average_pooling2d').output)
feature_extractor_pred = feature_extractor.predict(X_test)

In [ ]:
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
feature_extractor_pred = scaler.fit_transform(feature_extractor_pred)
feature_extractor_df = pd.DataFrame(feature_extractor_pred)
y_true_series = pd.Series(y_true)

classifier = XGBClassifier(random_state = 1)
classifier.fit(X =feature_extractor_df, y = y_true_series)
feature_importances = classifier.feature_importances_
feature_importances = pd.Series(feature_importances).sort_values(ascending= False)

In [ ]:
def plot_feature_importances(series, n_plotted): 
    series = series.sort_values(ascending = False).head(n_plotted)
    X = series
    y = series.index
    plt.figure(figsize = (15, 10))
    sns.barplot(x = X, y = y, order = y, orient = 'h', palette = 'viridis', hue = X, legend = False)
    
    plt.title(f'Top {n_plotted} Feature Importance Scores', fontsize = 14)
    plt.xlabel('Importance scores', fontsize = 14)
    plt.ylabel('Feature index', fontsize = 14)
    
    
    plt.tight_layout()
    plt.show()
    
plot_feature_importances(feature_importances, 50)

In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components = 2, random_state= 1, perplexity= 30, max_iter = 1000)
X_tsne_2d = tsne.fit_transform(feature_extractor_df)

plt.figure(figsize=(12, 8))
scatter = sns.scatterplot(x=X_tsne_2d[:, 0], y=X_tsne_2d[:, 1], hue=y_true, palette='viridis')
label_dict = {i: label for i, label in enumerate(class_labels)}

new_labels = [label_dict[int(item.get_text())] for item in scatter.legend_.texts]
for t, l in zip(scatter.legend_.texts, new_labels):
    t.set_text(l)

plt.title('t-SNE visualization of extracted features')
plt.xlabel('t-SNE feature 1')
plt.ylabel('t-SNE feature 2')
plt.tight_layout()
plt.show()

In [ ]:
tsne = TSNE(n_components=3, random_state=1, perplexity=30, max_iter =1000)
X_tsne_3d = tsne.fit_transform(feature_extractor_df)

fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')

scatter = ax.scatter(X_tsne_3d[:, 0], X_tsne_3d[:, 1], X_tsne_3d[:, 2], c=y_true, cmap='viridis')
ax.set_xlabel('t-SNE feature 1')
ax.set_ylabel('t-SNE feature 2')
ax.set_zlabel('t-SNE feature 3')
ax.set_title('3D t-SNE visualization of extracted features')

cbar = plt.colorbar(scatter)
cbar.set_ticks(range(len(class_labels)))
cbar.set_ticklabels(class_labels)

plt.tight_layout()
plt.show()

In [ ]:
import plotly.express as px

df = pd.DataFrame({
    'tsne-1': X_tsne_3d[:, 0],
    'tsne-2': X_tsne_3d[:, 1],
    'tsne-3': X_tsne_3d[:, 2],
    'class': [class_labels[i] for i in y_true]
})

fig = px.scatter_3d(df, x='tsne-1', y='tsne-2', z='tsne-3',
                    color='class', 
                    title='Interactive 3D t-SNE visualization of extracted features',
                    labels={'tsne-1': 't-SNE feature 1', 
                            'tsne-2': 't-SNE feature 2', 
                            'tsne-3': 't-SNE feature 3'},
                    hover_data=['class'])

fig.update_traces(marker=dict(size=5))
fig.update_layout(legend=dict(itemsizing='constant'))
fig.show()